In [1]:
from init import *

In [2]:
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/multi_index.hpp>

using namespace eosio;

#define N(name) name##_n.value


struct record {
   uint64_t    primary;
   uint128_t   secondary;
   uint64_t    data;
   uint64_t primary_key() const { return primary; }
   uint128_t get_secondary() const { return secondary; }
   EOSLIB_SERIALIZE( record, (primary)(secondary)(data))
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        name _self(receiver);

        if (N("sayhello") == action) {
            name code = name(receiver);
            uint64_t scope = "scope"_n.value;
            
            multi_index<"mytable4"_n,
                        record,
                        indexed_by< "bysecondary"_n,
                        const_mem_fun<record, uint128_t, &record::get_secondary> > > mytable( code, scope);
            
            auto itr = mytable.find(11);
            
            if (mytable.end() == itr) {
                mytable.emplace( _self, [&]( auto& row ) {
                    row.primary = 11;
                    row.secondary = 22;
                    row.data = 111;
                });
            } else {
                mytable.modify( itr, _self, [&]( auto& row ) {
                    row.data = 1111;
                    row.secondary = 44;
                });
            }
            
            itr = mytable.find(11);
            if (itr == mytable.end()) {
                mytable.emplace( _self, [&]( auto& row ) {
                    row.primary = 22;
                    row.secondary = 222;
                    row.data = 2222;
                });
            } else {
                print(itr->primary, " ", itr->secondary, " ", itr->data, "\n");
            }

            auto idx = mytable.get_index<"bysecondary"_n>();
            auto itr_sec = idx.find(222);
            print(itr_sec == idx.end(), "\n");
            print(itr_sec->primary, " ", itr_sec->secondary, " ", itr_sec->data, "\n");

            itr_sec = idx.find(888);
            print(itr_sec == idx.end(), "\n");
        }
    }
}
'''


code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [4]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

11 22 111
true
8496 0 0
true

1529


In [5]:
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/singleton.hpp>

using namespace eosio;

struct accountcounter {
   uint64_t                        count;
   EOSLIB_SERIALIZE( accountcounter, (count) )
};


 
typedef eosio::singleton< "global"_n, accountcounter >   account_counter;
  
extern "C" {
   void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
       account_counter counter(name(receiver), receiver);
       accountcounter a = {0};
       a = counter.get_or_default(a);
       a.count += 1;
       counter.set(a, name(receiver));
       print(a.count);
   }
}
'''

code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')


done!


In [6]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

1
1209


In [8]:
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/singleton.hpp>
#include <eosio/db.h>

using namespace eosio;

struct accountcounter {
   uint64_t                        count;
   EOSLIB_SERIALIZE( accountcounter, (count) )
};

typedef eosio::singleton< "global"_n, accountcounter >   account_counter;
  
extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if (0) {
            auto itr = db_find_i64(receiver, receiver, "global"_n.value, "global"_n.value);
            db_remove_i64(itr);
        }
        account_counter counter(name(receiver), receiver);
        accountcounter a = {0};
        a = counter.get_or_default(a);
        a.count += 1;
        counter.set(a, name(receiver));
        print(a.count, "\n");
    }
}

'''

code = uuosapi.compile(test_account1, code, vm_type=0)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [10]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

2

1071
